<a href="https://colab.research.google.com/github/SLCFLAB/Data-Science-Python/blob/main/Day%209/9_2_3_categorical_feature_model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 범주형 데이터 이진분류 경진대회 모델 성능 개선 II

- [범주형 데이터 이진분류 경진대회 링크](https://www.kaggle.com/c/cat-in-the-dat/)
- [모델링 코드 참고 링크](https://www.kaggle.com/dkomyagin/cat-in-the-dat-0-80285-private-lb-solution)

In [1]:
import pandas as pd

# 데이터 경로
data_path = 'https://raw.githubusercontent.com/SLCFLAB/Data-Science-Python/main/Day%209/data/'


train = pd.read_csv(data_path + 'cat_train.csv', index_col='id')
test = pd.read_csv(data_path + 'cat_test.csv', index_col='id')
submission = pd.read_csv(data_path + 'cat_sample_submission.csv', index_col='id')

## 피처 엔지니어링 I : 피처 맞춤 인코딩

### 데이터 합치기

In [2]:
# 훈련 데이터와 테스트 데이터 합치기
all_data = pd.concat([train, test])
all_data = all_data.drop('target', axis=1) # 타깃 값 제거

### 이진 피처 인코딩

In [3]:
all_data['bin_3'] = all_data['bin_3'].map({'F':0, 'T':1})
all_data['bin_4'] = all_data['bin_4'].map({'N':0, 'Y':1})

### 순서형 피처 인코딩

In [4]:
ord1dict = {'Novice':0, 'Contributor':1, 
            'Expert':2, 'Master':3, 'Grandmaster':4}
ord2dict = {'Freezing':0, 'Cold':1, 'Warm':2, 
            'Hot':3, 'Boiling Hot':4, 'Lava Hot':5}

all_data['ord_1'] = all_data['ord_1'].map(ord1dict)
all_data['ord_2'] = all_data['ord_2'].map(ord2dict)

In [5]:
from sklearn.preprocessing import OrdinalEncoder

ord_345 = ['ord_3', 'ord_4', 'ord_5']

ord_encoder = OrdinalEncoder() # OrdinalEncoder 객체 생성
# ordinal 인코딩 적용
all_data[ord_345] = ord_encoder.fit_transform(all_data[ord_345])

# 피처별 인코딩 순서 출력
for feature, categories in zip(ord_345, ord_encoder.categories_):
    print(feature)
    print(categories)

ord_3
['a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o']
ord_4
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']
ord_5
['AP' 'Ai' 'Aj' 'BA' 'BE' 'Bb' 'Bd' 'Bn' 'CL' 'CM' 'CU' 'CZ' 'Cl' 'DH'
 'DN' 'Dc' 'Dx' 'Ed' 'Eg' 'Er' 'FI' 'Fd' 'Fo' 'GD' 'GJ' 'Gb' 'Gx' 'Hj'
 'IK' 'Id' 'JX' 'Jc' 'Jf' 'Jt' 'KR' 'KZ' 'Kf' 'Kq' 'LE' 'MC' 'MO' 'MV'
 'Mf' 'Ml' 'Mx' 'NV' 'Nf' 'Nk' 'OR' 'Ob' 'Os' 'PA' 'PQ' 'PZ' 'Ps' 'QM'
 'Qb' 'Qh' 'Qo' 'RG' 'RL' 'RP' 'Rm' 'Ry' 'SB' 'Sc' 'TR' 'TZ' 'To' 'UO'
 'Uk' 'Uu' 'Vf' 'Vx' 'WE' 'Wc' 'Wv' 'XI' 'Xh' 'Xi' 'YC' 'Yb' 'Ye' 'ZR'
 'ZS' 'Zc' 'Zq' 'aF' 'aM' 'aO' 'aP' 'ac' 'av' 'bF' 'bJ' 'be' 'cA' 'cG'
 'cW' 'ck' 'cp' 'dB' 'dE' 'dN' 'dO' 'dP' 'dQ' 'dZ' 'dh' 'eG' 'eQ' 'eb'
 'eg' 'ek' 'ex' 'fO' 'fh' 'gJ' 'gM' 'hL' 'hT' 'hh' 'hp' 'iT' 'ih' 'jS'
 'jV' 'je' 'jp' 'kC' 'kE' 'kK' 'kL' 'kU' 'kW' 'ke' 'kr' 'kw' 'lF' 'lL'
 'll' 'lx' 'mb' 'mc' 'mm' 'nX' 'nh' 'oC' 'oG' 'oH' 'oK' 'od' 'on' 'pa'
 'ps' 'qA' 'qJ' 'qK' 'qP' 'qX' '

### 명목형 피처 인코딩

In [6]:
nom_features = ['nom_' + str(i) for i in range(10)] # 명목형 피처

In [7]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder() # OneHotEncoder 객체 생성
# 원-핫 인코딩 적용
encoded_nom_matrix = onehot_encoder.fit_transform(all_data[nom_features])

encoded_nom_matrix

<500000x16276 sparse matrix of type '<class 'numpy.float64'>'
	with 5000000 stored elements in Compressed Sparse Row format>

In [8]:
all_data = all_data.drop(nom_features, axis=1) # 기존 명목형 피처 삭제

### 날짜 피처 인코딩

In [9]:
date_features  = ['day', 'month'] # 날짜 피처

# 원-핫 인코딩 적용
encoded_date_matrix = onehot_encoder.fit_transform(all_data[date_features])

all_data = all_data.drop(date_features, axis=1) # 기존 날짜 피처 삭제

encoded_date_matrix

<500000x19 sparse matrix of type '<class 'numpy.float64'>'
	with 1000000 stored elements in Compressed Sparse Row format>

### 데이터 나누기

In [35]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X_train = all_data[:num_train] # 0 ~ num_train - 1행
X_test = all_data[num_train:] # num_train ~ 마지막 행

y = train['target'].values

# XGBoost 

XGBoost란 Extreme Gradient Boosting의 약자로, Gradient Boost를 병렬학습 가능하도록 구현함.

Regression과 Classification을 모두 지원하며, 성능과 효율이 좋은 편이다.





방법 1.  사이킷런 래퍼 XGBoost 모듈

In [36]:
import xgboost as xgb
from xgboost import XGBClassifier

# 1. 모델 선언
xgb_model  = XGBClassifier()

# 2. 모델 훈련 fit()함수
xgb_model.fit(X_train, y)

# 3. 모델 예측 predict()함수
Y_pred = xgb_model.predict(X_test)

# 4. score()
xgb_model.score(X_train, y)

0.7267766666666666

방법 2.  파이썬 래퍼 XGBoost 모듈

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, 
                                                      test_size=0.2, 
                                                      random_state=0)
dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_valid, y_valid)

In [54]:
params = {'max_depth':3,
         'eta':0.1,
         'objective':'binary:logistic',
         'eval_metric':'rmse',
         'early_stoppings':100}
num_rounds = 400
w_list = [(dtrain,'train'),(dvalid,'valid')]

In [55]:
xgb_ml = xgb.train(params = params, dtrain = dtrain, num_boost_round = num_rounds, early_stopping_rounds = 100, evals = w_list)

[0]	train-rmse:0.490052	valid-rmse:0.490094
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 100 rounds.
[1]	train-rmse:0.481789	valid-rmse:0.48181
[2]	train-rmse:0.474878	valid-rmse:0.474893
[3]	train-rmse:0.469073	valid-rmse:0.469141
[4]	train-rmse:0.464165	valid-rmse:0.464244
[5]	train-rmse:0.460051	valid-rmse:0.460172
[6]	train-rmse:0.45653	valid-rmse:0.456652
[7]	train-rmse:0.453548	valid-rmse:0.45364
[8]	train-rmse:0.450973	valid-rmse:0.451129
[9]	train-rmse:0.448801	valid-rmse:0.448993
[10]	train-rmse:0.446877	valid-rmse:0.447074
[11]	train-rmse:0.445217	valid-rmse:0.445422
[12]	train-rmse:0.443758	valid-rmse:0.444018
[13]	train-rmse:0.44248	valid-rmse:0.442759
[14]	train-rmse:0.441316	valid-rmse:0.441633
[15]	train-rmse:0.440263	valid-rmse:0.440581
[16]	train-rmse:0.439363	valid-rmse:0.439718
[17]	train-rmse:0.438542	valid-rmse:0.438909
[18]	train-rmse:0.437787	valid-rmse:0.438122
[19]	train-rm

In [60]:
# 혼동행렬, 정확도, 정밀도, 재현율, F1, AUC 불러오기
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score

def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))

In [57]:
import numpy as np
pred_probs = xgb_ml.predict(dvalid)
print('predict() 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨')
print(np.round(pred_probs[:10], 3))

# 예측 확률이 0.5보다 크면 1, 그렇지 않으면 0으로 예측값 결정해 리스트 객체인 preds에 저장
preds = [ 1 if x > 0.5 else 0 for x in pred_probs]
print('예측값 10개만 표시: ', preds[:10])

predict() 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨
[0.459 0.175 0.515 0.348 0.299 0.366 0.123 0.391 0.557 0.234]
예측값 10개만 표시:  [0, 0, 1, 0, 0, 0, 0, 0, 1, 0]


In [61]:
get_clf_eval(y_valid, preds)

오차행렬:
 [[37989  3689]
 [12771  5551]]

정확도: 0.7257
정밀도: 0.6008
재현율: 0.3030
F1: 0.4028
AUC: 0.6072
